# Лекция №5

## На прошлой лекции

[Процесс дискретизации](#discr)

[Метод контрольного объема (Finite Volume Method)](#FVM)

[Производная по времени](#ddt)

[Источник](#source)

[Операторы fvm и fvc в OpenFOAM](#fvof)

## План
1. [Диффузия](#lap)
2. [Интерполяция на грани](#inter_face)
3. [Качество сетки](#mesh_quality)
4. [Градиент на грани](#face_grad)
5. [OpenFOAM laplacian](#oflaplacian)
6. [СЛАУ](#slau)
7. [Заключение](#finish)

In [ ]:
#!cat $WM_PROJECT_DIR/applications/solvers/basic/laplacianFoam/laplacianFoam.C

```c++
/*---------------------------------------------------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     |
    \\  /    A nd           | www.openfoam.com
     \\/     M anipulation  |
-------------------------------------------------------------------------------
    Copyright (C) 2011-2017 OpenFOAM Foundation
    Copyright (C) 2019 OpenCFD Ltd.
-------------------------------------------------------------------------------
License
    This file is part of OpenFOAM.

    OpenFOAM is free software: you can redistribute it and/or modify it
    under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    OpenFOAM is distributed in the hope that it will be useful, but WITHOUT
    ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
    FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License
    for more details.

    You should have received a copy of the GNU General Public License
    along with OpenFOAM.  If not, see <http://www.gnu.org/licenses/>.

Application
    laplacianFoam

Group
    grpBasicSolvers

Description
    Laplace equation solver for a scalar quantity.

    \heading Solver details
    The solver is applicable to, e.g. for thermal diffusion in a solid.  The
    equation is given by:

    \f[
        \ddt{T}  = \div \left( D_T \grad T \right)
    \f]

    Where:
    \vartable
        T     | Scalar field which is solved for, e.g. temperature
        D_T   | Diffusion coefficient
    \endvartable

    \heading Required fields
    \plaintable
        T     | Scalar field which is solved for, e.g. temperature
    \endplaintable

\*---------------------------------------------------------------------------*/

#include "fvCFD.H"
#include "fvOptions.H"
#include "simpleControl.H"

// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

int main(int argc, char *argv[])
{
    argList::addNote
    (
        "Laplace equation solver for a scalar quantity."
    );

    #include "postProcess.H"

    #include "addCheckCaseOptions.H"
    #include "setRootCaseLists.H"
    #include "createTime.H"
    #include "createMesh.H"

    simpleControl simple(mesh);

    #include "createFields.H"

    // * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

    Info<< "\nCalculating temperature distribution\n" << endl;

    while (simple.loop())
    {
        Info<< "Time = " << runTime.timeName() << nl << endl;

        while (simple.correctNonOrthogonal())
        {
            fvScalarMatrix TEqn
            (
                fvm::ddt(T) - fvm::laplacian(DT, T)
             ==
                fvOptions(T)
            );

            fvOptions.constrain(TEqn);
            TEqn.solve();
            fvOptions.correct(T);
        }

        #include "write.H"

        runTime.printExecutionTime(Info);
    }

    Info<< "End\n" << endl;

    return 0;
}


// ************************************************************************* //
```

## Метод контрольного объема (Finite Volume Method) <a name="FVM"></a>

<img src="img/finite_volume.png" width="1000">

Уравнение переноса в интегральной форме:

$$
\int_V {\partial{\rho \phi}\over {\partial t}} dV + \int_V \nabla \cdot \left(\rho \phi \vec{U} \right) dV = \int_V \nabla \cdot \left(\Gamma \nabla \phi \right) dV + \int_V S_\phi dV
$$

$dV$ - элементарный объем (ячейка сетки) "картинка ячейки"

$\phi$ - объемное поле (volScalarField) ($\rho,\rho U,\rho E, T$)

$\vec U$ - объемное поле скорости

$\rho$ - объемное поле плотности

$\Gamma$ - объемное поле коэффициента диффузии

$S_\phi$ - объемное поле источника

<img src="img/cell.png" width="300">

[FVM wolfdynamics](../books/wd_fvm.pdf) p.12

[ProgrammerGuide](../books/OFpg.pdf) p.25

<img src="img/ddt.jpg" height="300">

Производная по времени:

$$\int_V {\partial{\rho \phi}\over {\partial t}} dV \approx ({\partial{\rho \phi}\over {\partial t}})_P V_p$$

<img src="img/source_beer.jpg" height="300">

Источник:
$$\int_V S_\phi dV \approx S_{\phi P} V_P$$

Источник и производная по времени: только объемные поля (volScalarField), нет связи с соседними ячейками

Теорема Остроградского-Гаусса:

$$\int_V \left( \nabla \cdot \vec u \right) dV = \oint_S \left( \vec{n} \cdot \vec u \right) dS \approx
\sum\limits_{face} \left( \vec{n} \cdot \vec u \right)_f S_f
$$

Конвекция:

$$\int_V \nabla \cdot \left(\rho \phi \vec{U} \right) dV \approx
\sum\limits_{face} \left(\rho \phi \vec{n} \cdot \vec U \right)_f S_f$$

Диффузия:

$$\int_V \nabla \cdot \left(\Gamma \nabla \phi \right) dV \approx
\sum\limits_{face} \left(\Gamma \nabla \phi  \cdot \vec{n} \right)_f S_f$$

Конвекция и диффузия: поля на гранях (surfaceScalarField), связь с соседними ячейками

## Диффузия <a name="lap"></a>

<img src="img/laplace_stakan.jpeg" width="300">

Переливание от большего к меньшему!

$$\int_V \nabla \cdot \left(\Gamma \nabla \phi \right) dV = 
\oint_S \left(\Gamma \nabla \phi  \cdot \vec{n} \right) dS \approx
\sum\limits_{face} \left(\Gamma \nabla \phi  \cdot \vec{n} \right)_f S_f$$

поля на гранях (surfaceScalarField)

[OpenFOAM laplacian](https://www.openfoam.com/documentation/guides/latest/doc/guide-schemes-laplacian.html)

[ProgrammerGuide](../books/OFpg.pdf) p.36

[UserGuide](../books/OFug.pdf) p.120

### Интерполяция на грани  <a name="inter_face"></a>

$$\int_V \nabla \cdot \left(\Gamma \nabla \phi \right) dV = 
\oint_S \left(\Gamma \nabla \phi  \cdot \vec{n} \right) dS \approx
\sum\limits_{face} \left(\Gamma \nabla \phi  \cdot \vec{n} \right)_f S_f$$
$$\Gamma_V \rightarrow \Gamma_f$$ 
volScalarField -> surfaceScalarField

<img src="img/cell.png" width="300">

In [ ]:
!ls $WM_PROJECT_DIR/src/finiteVolume/interpolation

In [ ]:
!ls $WM_PROJECT_DIR/src/finiteVolume/interpolation/interpolation

In [ ]:
!ls $WM_PROJECT_DIR/src/finiteVolume/interpolation/surfaceInterpolation

In [ ]:
!cat $WM_PROJECT_DIR/src/finiteVolume/interpolation/surfaceInterpolation/surfaceInterpolationScheme/surfaceInterpolationScheme.C

In [ ]:
!ls $WM_PROJECT_DIR/src/finiteVolume/interpolation/surfaceInterpolation/schemes

In [ ]:
!cat $WM_PROJECT_DIR/src/finiteVolume/interpolation/surfaceInterpolation/schemes/harmonic/harmonic.H

In [ ]:
!cat $WM_PROJECT_DIR/src/finiteVolume/interpolation/surfaceInterpolation/schemes/midPoint/midPoint.H

## Качество сетки <a name="mesh_quality"></a>

[Mesh wolfdynamics](../books/wd_mesh.pdf) p.22

* Orthogonality.
* Skewness.
* Aspect Ratio.
* Smoothness.

#### Orthogonality.

$$cos(\theta)$$

Угол между вектором нормали (грани) и вектором центров ячеек

Влияет на вычисление градиента на грани (на диффузию)

"Добавляет вязкости"

<img src="img/orth.png" width="500">

#### Skewness.

$$\Delta_i$$

Влияет на интерполяцию

Добавляет вязкости и нестабильности

Влияет на конвектиный и диффузионный члены

<img src="img/skew.png" width="500">

#### Aspect Ratio.

$$\Delta y / \Delta x$$

Влияет на расчет градиентов

Жесткость СЛАУ

<img src="img/ar.png" width="500">

#### Smoothness.

Точность диффузионного слагаемого

<img src="img/smooth.png" width="500">

In [ ]:
!ls mesh_quality

<img src="img/mesh.png" width="500">

In [ ]:
!checkMesh -case mesh_quality -allTopology -allGeometry

## Возможности учета "качества сетки" в OpenFOAM:

* коррекция в операторах (неортогональность, скошенность) дивергенции (fvc::laplace, fvc::div, fvm::laplace, fvm:div), схемы в fvSchemes (corrected, limited, skew...) 
* с версии 2012 [коррекция расчета центров ячеек и граней](https://www.openfoam.com/news/main-news/openfoam-v20-12/numerics) (опция в fvSchemes)

[Подробнее про расчет центров ячеек](../books/Nishikawa_centroid_skew_correct.pdf)

Nishikawa - много материалов по CFD

[Nishikawa researchgate](https://www.researchgate.net/profile/Hiroaki-Nishikawa-2)

[Nishikawa site](http://www.hiroakinishikawa.com/)

### Градиент на грани <a name="face_grad"></a>

$$\int_V \nabla \cdot \left(\Gamma \nabla \phi \right) dV = 
\oint_S \left(\Gamma \nabla \phi  \cdot \vec{n} \right) dS \approx
\sum\limits_{face} \left(\Gamma \nabla \phi  \cdot \vec{n} \right)_f S_f$$

[Surface-normal gradient schemes](https://www.openfoam.com/documentation/guides/latest/doc/guide-schemes-sngrad.html)

$$\nabla Q = \vec{n} \cdot \left( \nabla Q \right)_f$$

<img src="img/schemes-sngrad-schematic.png" width="300">

* [Orthogonal surface-normal gradient scheme](https://www.openfoam.com/documentation/guides/latest/doc/guide-schemes-sngrad-orthogonal.html)
* [Uncorrected surface-normal gradient scheme](https://www.openfoam.com/documentation/guides/latest/doc/guide-schemes-sngrad-uncorrected.html)
* [Corrected surface-normal gradient scheme](https://www.openfoam.com/documentation/guides/latest/doc/guide-schemes-sngrad-corrected.html)
* [Limited surface-normal gradient scheme](https://www.openfoam.com/documentation/guides/latest/doc/guide-schemes-sngrad-limited.html)

```c++
snGradSchemes
{
    default         none;
}
```

[Orthogonal surface-normal gradient scheme](https://www.openfoam.com/documentation/guides/latest/doc/guide-schemes-sngrad-orthogonal.html)

$$\nabla Q = \frac{Q_P - Q_N}{| \vec{d} |}$$

<img src="img/schemes-sngrad-orthogonal.png" width="300">

```c++
snGradSchemes
{
    default         none;
    snGrad(Q)       orthogonal;
}
```

[Uncorrected surface-normal gradient scheme](https://www.openfoam.com/documentation/guides/latest/doc/guide-schemes-sngrad-uncorrected.html)

$$\nabla Q = \alpha \frac{Q_P - Q_N}{| \vec{d} |} $$
$$\alpha = \frac{1}{cos(\theta)}$$

<img src="img/schemes-sngrad-schematic.png" width="300">

```c++
snGradSchemes
{
    default         none;
    grad(U)         uncorrected;
}
```

[Corrected surface-normal gradient scheme](https://www.openfoam.com/documentation/guides/latest/doc/guide-schemes-sngrad-corrected.html)

$$\nabla Q = \underbrace{\alpha \frac{Q_P - Q_N}{| \vec{d} |}}_{\mathrm{implicit}} + \underbrace{\left( \hat{\vec{n}} - \alpha \hat{\vec{d}} \right) \cdot \left( \nabla Q \right)_f}_{\mathrm{explicit\ correction}}$$

$$\alpha = \frac{1}{cos(\theta)}$$

<img src="img/schemes-sngrad-schematic.png" width="300">

```c++
snGradSchemes
{
    default         none;
    grad(U)         corrected;
}
```

[Limited surface-normal gradient scheme](https://www.openfoam.com/documentation/guides/latest/doc/guide-schemes-sngrad-limited.html)

$$\nabla Q = \underbrace{\alpha \frac{Q_P - Q_N}{| \vec{d} |}}_{\mathrm{implicit}} + \underbrace{\psi * \left( \hat{\vec{n}} - \alpha \hat{\vec{d}} \right) \cdot \left( \nabla Q \right)_f}_{\mathrm{explicit\ correction}}$$

$$\alpha = \frac{1}{cos(\theta)}$$

<img src="img/schemes-sngrad-schematic.png" width="300">

```c++
snGradSchemes
{
    default         none;
    grad(U)         limited <corrected scheme> <psi>;
}
psi coefficient between 0 and 1

0: no correction, equivalent to the uncorrected scheme
1: full correction applied
0.5: non-orthogonal contribution does not exceed the orthogonal part
```

$$\nabla Q = \underbrace{\alpha \frac{Q_P - Q_N}{| \vec{d} |}}_{\mathrm{implicit}} + \underbrace{\left( \hat{\vec{n}} - \alpha \hat{\vec{d}} \right) \cdot \left( \nabla Q \right)_f}_{\mathrm{explicit\ correction}}$$

<img src="img/mesh_slau.png" width="500">

In [ ]:
!ls $WM_PROJECT_DIR/src/finiteVolume/finiteVolume/laplacianSchemes

In [ ]:
!cat $WM_PROJECT_DIR/src/finiteVolume/finiteVolume/laplacianSchemes/gaussLaplacianScheme/gaussLaplacianScheme.C

[lduMatrix doxygen](https://www.openfoam.com/documentation/guides/latest/api/classFoam_1_1lduMatrix.html)

Важное свойство - коэффициент на диагонали равен сумме недиагональных коэффициентов

$ \phi $ - поле (неизвестная величина на новом слое по времени)

$ \Gamma $ - известное поле коэффициента диффузии, часто зависит от $\phi$

$ S_f $ - площадь грани, const, меняется только при движение узлов сетки

$ \vec{n} $ - нормаль грани, const, меняется только при движение узлов сетки

$$\sum\limits_{face} \left(\Gamma \nabla \phi  \cdot \vec{n} \right)_f S_f$$

$\sum\limits_{face} \left(\Gamma^n \nabla \phi^n  \cdot \vec{n}^n \right)_f S_f^n$ - полностью явно

$\sum\limits_{face} \left(\Gamma^n \nabla \phi^{n+1}  \cdot \vec{n}^n \right)_f^n S_f^n$ - неявно

$\sum\limits_{face} \left(\Gamma^{n+1} \nabla \phi^{n+1}  \cdot \vec{n}^{n+1} \right)_f S_f^{n+1}$ - полностью неявно только итерационно (подцикл на текущем шаге по времени - простая итерация или метод Ньютона)

слой по времени для полей (явная или неявная схема) задается в уравнение непосредственно операторами: 

* fvm - неявная схема $\phi^{n+1} $
* fvc - явная схема $\phi^{n} $

Все вспомогательные поля ($\Gamma, S_f, \vec{n}$) могут использоваться только с текущего (известного слоя по времени)

Жесткость системы: $\Gamma_f S_f 1/ d$

Смысл (если $\Gamma > 0$): $\phi$ перетекает от большего к меньшему 

Важно: 2я производная по пространству - жесткое ограничение на число Куранта при измельчении сетки! $\Delta t \sim \Delta x^2$

Консервативность - потоки с обратным знаком (нормаль меняет знак для ячейки владельца и соседа)

Только соседи - низкий порядок точности. Как повысить? См. WENO схемы.

## OpenFOAM laplacian <a name="oflap"></a>


[ProgrammerGuide](../books/OFpg.pdf) p.35

[FiniteVolume OF Doxygen](https://www.openfoam.com/documentation/guides/latest/api/namespaceFoam_1_1fv.html)

В OpenFOAM существуют стандартные операторы производных: $\frac{\partial }{\partial t},\frac{\partial ^2}{\partial
t^2},(\nabla \cdot \ ),\nabla $, которые возвращают коэффициенты СЛАУ вида:  $A\vec x^{n+1}=\vec b(\vec x^n)$, где 
$\vec x^{n+1}$ - поле искомой переменной (например  $T$ ) на новом слое по времени ,  $\vec b(\vec x^n)$ - правая часть
СЛАУ (входят все величины, зависящие от предшествующих шагов по времени:  $n,n-1...$)

fvm — неявная аппроксимация (возвращает коэффициенты системы линейных уравнений  $A$ и $\vec b$).

fvc — явная аппроксимация (возвращается вектор правой части СЛАУ -  $\vec b$).

Пример кода, использования операторов производной по времени и лапласиана:

$$\frac{\partial T}{\partial t} + \nabla \cdot (\Gamma \nabla T) = 0 $$

```c++
while(runTime.run()) // цикл по времени (заканчивается по критерию остановки
{
runTime++; // переход к следующей итерации по времени 
    
//TEqn — СЛАУ для поля  T (volScalarField)

//fvm::ddt(T) — аппроксимация производной по времени

//fvm::laplacian(Gamma, T) — неявная аппроксимация лапласиана, Gamma - объемное поле volScalarField

//fvс::laplacian(Gamma, T) — явная аппроксимация лапласиана, Gamma - объемное поле volScalarField

fvScalarMatrix TEqn (                                        //- инициализация СЛАУ
                    fvm::ddt(T)                              // производная по времени
                    + theta*fvm::laplacian(Gamma, T, "implicit_laplace")         // неявная часть оператора лапласса
                    + (1.0-theta)*fvс::laplacian(Gamma, T, "explicit_laplace")   // явная часть оператора лапласса
                    ); 

TEqn.relax(); // Релаксация (необходима в случае стационарной задачи), см ниже

TEqn.solve();// - решение СЛАУ (результат записывается в T!!!)

runTime.write(); // запись всех полей из регистра
}
```

fvm::laplacian(Gamma, T) - возвращает A (и b если коррекция)

fvc::laplacian(Gamma, T) - возвращает b 

Все схемы для аппроксимации производных задаются в файле system/fvSchemes
```c++
ddtSchemes
{
    default Euler; // backward;
}

laplacianSchemes
{
    default         none;
    laplacian(Gamma,T) Gauss linear limited 1;
    implicit_laplace   Gauss linear limited 1;
    explicit_laplace   Gauss linear limited 1;
}

interpolationSchemes
{
    default         linear;
}

snGradSchemes
{
    default         limited 1;
}
```

Метод решения СЛАУ задается в файле system/fvSolution
```c++
solvers
{
    T
    {
        solver          PCG;
        preconditioner  DIC;
        tolerance       1e-06;
        relTol          0;
    }
}

SIMPLE
{
    nNonOrthogonalCorrectors 2;
}
```

In [ ]:
# tut/basic/laplacianFoam/flange
!flange/Allclean
!ls flange

In [ ]:
!cat flange/system/controlDict

In [ ]:
!cat flange/system/fvSchemes

In [ ]:
!cat flange/system/fvSolution

In [ ]:
!cat flange/constant/transportProperties

In [ ]:
!cat flange/0/T

In [ ]:
!cat flange/Allrun

In [ ]:
!flange/Allrun

In [ ]:
!ls flange

In [ ]:
!checkMesh -case flange

In [ ]:
!cat flange/log.laplacianFoam

<table><tr>
<td> <img src="img/mesh_flange.png" width="400"> </td>
<td> <img src="img/field_T_flange_0.1.png" width="400"> </td>
<td> <img src="img/field_T_flange.png" width="400"> </td>
</tr></table>

In [ ]:
%%HTML
<video width="1080" height="720" controls>
  <source src="img/output.mp4" type="video/mp4">
</video>

## СЛАУ <a name="slau"></a>

[OpenFOAM СЛАУ doxygen](https://www.openfoam.com/documentation/guides/latest/doc/guide-solvers.html)

СЛАУ:

$$
\begin{bmatrix} a_{11} & a_{12} & \dots & a_{1m} \\ a_{21} & a_{22} & \dots & a_{2m} \\ \vdots & \vdots & \ddots & \vdots \\ a_{m1} & a_{m2} & \dots & a_{mm} \end{bmatrix} \begin{bmatrix} \phi_{1}^{n+1} \\ \phi_{2}^{n+1} \\ \vdots \\ \phi_{m}^{n+1} \end{bmatrix} = \begin{bmatrix} b_{1} \\ b_{2} \\ \vdots \\ b_{m} \end{bmatrix}
$$

$n$ - индекс по времени

$m$ - номер ячейки

$$
A \vec{\phi}^{n+1} = \vec{b}
$$
$A$ - коэффициенты матрицы

$\vec{\phi}^{n+1}$ - неизвестное объемное поле величины

$\vec{b}(\phi^{n, n-1 ...}) $ - правая часть (объемное поле)

Явные схемы - все операторы зависят только от $\phi_n$ - диагональная матрица $A$
$$\vec{\phi}^{n+1} = A^{-1} \vec{b}$$
$${A}^{-1} = \frac{1}{\mathrm{diag}({A})}$$

Неявные схемы - линейная связь неизвестного значения на новом слое с соседями!

Соседних ячеек мало (кол-во граней ячейки), общее кол-во ячеек огромно - разреженная матрица! 

Большая часть коэффициентов равна нулю

[LduMatrix doxygen](https://www.openfoam.com/documentation/guides/latest/api/classFoam_1_1LduMatrix.html)

LDU формат хранения матрицы

Явная связь топологии сетки (связи между ячейками) с ненулевыми коэффициентами матрицы!

Основа - грань (две ячейки - владелец и сосед)

L - owner (владельцы)

U - neighbour (соседи)

<img src="img/mesh_slau.png" width="500">

### Итерационные методы решения СЛАУ:

[OpenFOAM darwish](../books/darwish.pdf) p.328 (303)

[Лекции Кузнецов](https://vk.com/slae2021)

1. [Smooth solvers](https://www.openfoam.com/documentation/guides/latest/doc/guide-solvers-smooth.html) (предобуславливатели)
2. [Conjugate gradient solvers](https://www.openfoam.com/documentation/guides/latest/doc/guide-solvers-cg.html)
3. [Multigrid solvers](https://www.openfoam.com/documentation/guides/latest/doc/guide-solvers-multigrid.html)


### [Smooth solvers](https://www.openfoam.com/documentation/guides/latest/doc/guide-solvers-smooth.html) Предобуславливатели

Предобуславливание - уменьшение числа обусловленности задачи ($\mu$, $\lambda_{max}/\lambda_{min}$)

Ошибки есть всегда! $A = A+\delta A, \vec b = \vec b + \vec {\delta b}$

Неидеальность дескретизации (сетка, операторы)

Симметричные (диффузия) и несимметричные (конвекция) матрицы

* DIC Gauss Seidel Smoother
* DIC Smoother - Предобуславливатель Холецкого
* FDIC Smoother
* DILU Smoother
* Gauss Seidel Smoother
* Symmetric Gauss Seidel Smoother

fvSolution
```c++
{
solver          smooth;
smoother        <smoother>;
relTol          <relative tolerance>; // относетльная невязка (до какой точности решать на текущем шаге)
tolerance       <absolute tolerance>; // общая невязка (до какой точности решать в целом)
}
```

<img src="img/slau_error.png" width="300">

### [Conjugate gradient solvers](https://www.openfoam.com/documentation/guides/latest/doc/guide-solvers-cg.html)

Градиентные методы, стандартный набор

* Preconditioned bi-conjugate gradient (PBiCG)
* Preconditioned bi-conjugate gradient (PBiCGStab)
* Preconditioned conjugate gradient (PCG)

fvSolution
```c++
{
solver          PBiCG;
preconditioner  <conditioner>;
relTol          <relative tolerance>;
tolerance       <absolute tolerance>;
}
```
    
GMRES (нет в OpenFOAM v2106..., есть в extended версии)

### [Multigrid solvers](https://www.openfoam.com/documentation/guides/latest/doc/guide-solvers-multigrid.html)

Алгебраический многосеточный метод (GAMG)

<img src="img/mesh_gamg.png" height="300" >
<img src="img/restrict_step.png" height="300" >
<img src="img/prolong_step.png" height="300">

Меньше памяти (по сравнению с PCG), используется на GPU

fvSolution
```c++
{
// Mandatory entries
solver                  GAMG;
smoother                <smoother>;
relTol                  <relative tolerance>;
tolerance               <absolute tolerance>;


// Optional entries and associated default values

// Agglomeration
cacheAgglomeration      yes;
nCellsInCoarsestLevel   10;
processorAgglomerator   <processor agglomeration method>;

// Solver
nPreSweeps              0;
preSweepsLevelMultiplier 1;
maxPreSweeps            4;
nPostSweeps             2;
postSweepsLevelMultiplier 1;
maxPostSweeps           4;
nFinestSweeps           2;
interpolateCorrection   no;
scaleCorrection         yes;  // Yes: symmetric No: Asymmetric
directSolveCoarsest     no;
```

### OpenFOAM контроль решения
* [Under relaxation](https://www.openfoam.com/documentation/guides/latest/doc/guide-solvers-under-relaxation.html) релаксация
* [Residuals](https://www.openfoam.com/documentation/guides/latest/doc/guide-solvers-residuals.html) невязка
* [Case termination](https://www.openfoam.com/documentation/guides/latest/doc/guide-solvers-case-termination.html)

### [Under relaxation](https://www.openfoam.com/documentation/guides/latest/doc/guide-solvers-under-relaxation.html)

Field under-relaxation $T^n = T^{n-1} + \alpha \left( T^n - T^{n-1} \right)$ 

T.relax() // volSalarField T

Equation under-relaxation

Стационарная постановка задачи (интересно решение при $\lim_{t \to \infty}$ ) $$\nabla \cdot (\Gamma \nabla T) = 0 $$

На самом деле решается уравнение с производной по времени (метод установления)

Ускорение за счет локального шага по времени или релаксации (одно и тоже)

$\lim_{n \to \infty} T^n_j \approx T^{n-1}_j $ , для всех j, где j - номер ячейки

$\beta_j T^n_j = \beta_j T^{n-1}_j $

!ВНИМАНИЕ, $\beta_j$ - свое для каждой ячейки 

Можно интерпретировать как локальный шаг по времени для каждой ячейки: ${\partial{T} \over {\partial t}} = \beta (T^n - T^{n-1}) = {1 \over {\Delta t}}(T^n - T^{n-1}) $

$ \alpha = {\sum a} / d $

Диагональное преобладание (диагональный элемент > суммы недиагональных)

Используется: steadyState (time scheme fvSchemes) + eq relax. Только для стационарных задач и только в связке с релаксацией

steadyState схема - ничего не делает (коэффициент на диагонали - 0 в случае оператора fvm::ddt)

TEq.relax() // fvScalarMatrix

fvSolution
```c++
relaxationFactors
{
    fields
    {
     T alpha;
    }
    equations
    {
     T alpha;
    }
}
```

$\alpha = 0$ - старый шаг ( 0 на диагонали) - меньше стабильность, больше итераций, меньше время одной итерации

$\alpha = 1$ - новый шаг (1 на диагонали) - больше стабильность, меньше итераций, больше время одной итерации

обычно выбирают 0.7-0.9 для более быстрого решения

Внимание!

fvm::laplacian(Gamma, T) + fvm::SuSp(T)

это не одно и тоже

-fvm::laplacian(Gamma, T) - fvm::SuSp(T)

Релаксация - положительные коэффициенты на диагонали. НЕТ СИММЕТРИИ относительно знака слагаемых в уравнение!

### [Residuals](https://www.openfoam.com/documentation/guides/latest/doc/guide-solvers-residuals.html)

```c++
tolerance       1e-6;
relTol          0.1;
```
СЛАУ:
$${A} \vec{x} = \vec{b}$$
Невязка:
$$\vec{r} = \vec{b} - {A} \vec{x}$$

Норма:
$$n = \sum \left( |{{A}\vec{x} - {A}\overline{\vec{x}}}| + |{\vec{b} - {A}\overline{\vec{x}}}| \right)$$

Итог:
$$r = \frac{1}{n} \sum |{\vec{b} - {A} \vec{x}}|$$

### [Case termination](https://www.openfoam.com/documentation/guides/latest/doc/guide-solvers-case-termination.html)

дополнительные настройки в fvSolution

#### Стационарные задачи

SIMPLE алгоритм

Контроль завершения программы
```c++
residualControl
{
    p           1e-2;
    "(Ux Uy)"   1e-4;
    "(k|epsilon|omega)" 1e-4;
}
```

#### Нестационарные задачи

PIMPLE алгоритм

```c++
// Maximum number of outer correctors
nOuterCorrectors    50;

residualControl
{
    "(U|k|epsilon|omega)"
    {
        relTol          0;
        tolerance       1e-4;
    }
}
```

## Заключение <a name="finish"></a>

1. [Диффузия](#lap)
2. [Интерполяция на грани](#inter_face)
3. [Качество сетки](#mesh_quality)
4. [Градиент на грани](#face_grad)
5. [OpenFOAM laplacian](#oflaplacian)

Для зачета написать программу:

Задача Коши для уравнения теплопроводности с источниковым членом на отрезке [0,L]:

$$\frac{\partial T}{\partial t} = \nabla \cdot (\Gamma \nabla T) + \alpha T^k $$

$$ T(t=0) = 0$$

$$ T(x=0) = T_0*(2+sin(\omega t))/2$$
$$ \nabla T(x=L) = 0$$

$$\Gamma, \alpha,k,T_0, \omega = const$$

2 балла. 2 вопроса. Каждую неделю кол-во вопросов увеличивается в 2 раза.

Совет

[ГУ sin в OpenFOAM - Function1](https://www.openfoam.com/documentation/guides/latest/api/classFoam_1_1Function1.html)

Пример использования ГУ - tutorials/compressible/rhoPimpleFoam/laminar/sineWaveDamping/0/p

Следите за числом Куранта! Можно прямо в программе посчитать!

## В следующей серии ...

Сдача задания!

<table><tr>
<td> <img src="img/zach_1.jpeg" width="400"> </td>
<td> <img src="img/zach_2.jpeg" width="400"> </td>
<td> <img src="img/zach_3.jpeg" width="400"> </td>
</tr></table>
